In [8]:
import tensorflow as tf
import numpy as np
from sklearn import datasets, metrics, preprocessing

boston = datasets.load_boston()
x_data = preprocessing.StandardScaler().fit_transform(boston.data)
y_data = boston.target
y_data = np.expand_dims(y_data, -1)

with tf.name_scope('params'):
    w = tf.Variable(tf.random_normal((13,1)), name='w', dtype=tf.float32)
    b = tf.Variable(0, name='b', dtype=tf.float32)
    
with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, shape=(None,13), name='x')
    y = tf.placeholder(tf.float32, shape=(None, 1), name='y')    
    
with tf.name_scope('estimation'):
    y_pred = tf.matmul(x, w) + b
    loss = tf.reduce_mean(tf.square(y - y_pred))    
    
    learning_rate = 0.1
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

epochs = 200
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        _, mse = sess.run([train_step, loss], feed_dict={
            x: x_data,
            y: y_data
        })
    print(mse)


21.9005


In [6]:
import tensorflow as tf
from tensorflow.contrib import learn
from sklearn import datasets, metrics, preprocessing

boston = datasets.load_boston()
x_data = preprocessing.StandardScaler().fit_transform(boston.data)

feature_columns = learn.infer_real_valued_columns_from_input(x_data)

reg = learn.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.1)    
)
reg.fit(x_data, boston.target, steps=200, batch_size=506)
result = reg.evaluate(x_data, boston.target, steps=1)
print(result['loss'])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0439c4bdd8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpp2m48_qq'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpp2m48_qq/mo

In [7]:
import sys
import numpy as np
from tensorflow.contrib import learn
from tensorflow.examples.tutorials.mnist import input_data

DATA_DIR = '/tmp/data'
data = input_data.read_data_sets(DATA_DIR, one_hot=False)
x_data, y_data = data.train.images, data.train.labels.astype(np.int32)
x_test, y_test = data.test.images, data.test.labels.astype(np.int32)

NUM_STEPS = 2000
MINIBATCH_SIZE = 128
feature_columns = learn.infer_real_valued_columns_from_input(x_data)

dnn = learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[200],
    n_classes=10,
    optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.2)
)
dnn.fit(x=x_data, y=y_data, steps=NUM_STEPS, batch_size=MINIBATCH_SIZE)

acc = dnn.evaluate(x=x_test, y=y_test, steps=1)['accuracy']
print('Test accuracy: {}'.format(acc))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please switch to

In [9]:
from sklearn.metrics import confusion_matrix

y_pred = dnn.predict(x=x_test, as_iterable=False)
class_names = ['0','1','2','3','4','5','6','7','8','9']
cnf_matrix = confusion_matrix(y_test, y_pred)

print(cnf_matrix)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp57pey9oa/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[ 973    1    1    1    0    0    2    1    1    0]
 [   0 1125    2    1    0    1    3    1    2    0]
 [   9    4  996    3    2    0    6    5    7    0]
 [   1    0    6  993    0    1    0    3    3    3]
 [   3    0    2    1  963    0    5    4    0    4]
 [   4    0    0   13    1  859    8    1    5    1]
 [   5    3    0    1    3    1  945    0    0    0]
 [   2    5    8    2    1    0    0 1002    2    6]
 [   3    0    4    6    4    4    7    2  943    1]
 [   6    6    0   10   12    1    0    8    2  964]]


In [13]:
import pandas as pd
N = 10000

weight = np.random.randn(N) * 5 + 70
spec_id = np.random.randint(0, 3, N)
bias = [0.9, 1.0, 1.1]
height = np.array([weight[i] / 100 + bias[b] for i, b in enumerate(spec_id)])
spec_name = ['Goblin', 'Human', 'ManBears']
spec = [spec_name[s] for s in spec_id]

df = pd.DataFrame({ 
    'Species': spec, 'Weight': weight, 'Height': height 
})

from tensorflow.contrib import layers

Weight = layers.real_valued_column('Weight')
Species = layers.sparse_column_with_keys(column_name='Species', keys=['Goblin', 'Human', 'ManBears'])
reg = learn.LinearRegressor(feature_columns=[Weight, Species])

def input_fn(df):
    feature_cols = {}
    feature_cols['Weight'] = tf.constant(df['Weight'].values)
    feature_cols['Species'] = tf.SparseTensor(
        indices=[[i, 0] for i in range(df['Species'].size)],
        values=df['Species'].values,
        dense_shape=[df['Species'].size, 1]
    )
    labels = tf.constant(df['Height'].values)
    return feature_cols, labels
    
reg.fit(input_fn=lambda:input_fn(df), steps=50000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f04317b7550>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmp991ch_v0'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp991ch_v0/mo

INFO:tensorflow:global_step/sec: 646.344
INFO:tensorflow:loss = 0.000533804, step = 7201 (0.154 sec)
INFO:tensorflow:global_step/sec: 628.305
INFO:tensorflow:loss = 0.000518131, step = 7301 (0.159 sec)
INFO:tensorflow:global_step/sec: 600.69
INFO:tensorflow:loss = 0.000502918, step = 7401 (0.166 sec)
INFO:tensorflow:global_step/sec: 598.68
INFO:tensorflow:loss = 0.000488152, step = 7501 (0.166 sec)
INFO:tensorflow:global_step/sec: 610.688
INFO:tensorflow:loss = 0.00047382, step = 7601 (0.166 sec)
INFO:tensorflow:global_step/sec: 641.155
INFO:tensorflow:loss = 0.000459908, step = 7701 (0.155 sec)
INFO:tensorflow:global_step/sec: 741.267
INFO:tensorflow:loss = 0.000446405, step = 7801 (0.135 sec)
INFO:tensorflow:global_step/sec: 735.244
INFO:tensorflow:loss = 0.000433299, step = 7901 (0.136 sec)
INFO:tensorflow:global_step/sec: 735.229
INFO:tensorflow:loss = 0.000420577, step = 8001 (0.136 sec)
INFO:tensorflow:global_step/sec: 694.143
INFO:tensorflow:loss = 0.000408228, step = 8101 (0.14

INFO:tensorflow:global_step/sec: 743.66
INFO:tensorflow:loss = 4.7764e-05, step = 15301 (0.135 sec)
INFO:tensorflow:global_step/sec: 617.273
INFO:tensorflow:loss = 4.63618e-05, step = 15401 (0.163 sec)
INFO:tensorflow:global_step/sec: 707.819
INFO:tensorflow:loss = 4.50007e-05, step = 15501 (0.141 sec)
INFO:tensorflow:global_step/sec: 606.63
INFO:tensorflow:loss = 4.36796e-05, step = 15601 (0.172 sec)
INFO:tensorflow:global_step/sec: 323.768
INFO:tensorflow:loss = 4.23973e-05, step = 15701 (0.302 sec)
INFO:tensorflow:global_step/sec: 557.404
INFO:tensorflow:loss = 4.11526e-05, step = 15801 (0.179 sec)
INFO:tensorflow:global_step/sec: 656.384
INFO:tensorflow:loss = 3.99444e-05, step = 15901 (0.152 sec)
INFO:tensorflow:global_step/sec: 357.484
INFO:tensorflow:loss = 3.87716e-05, step = 16001 (0.283 sec)
INFO:tensorflow:global_step/sec: 549.25
INFO:tensorflow:loss = 3.76335e-05, step = 16101 (0.179 sec)
INFO:tensorflow:global_step/sec: 733.764
INFO:tensorflow:loss = 3.65286e-05, step = 16

INFO:tensorflow:global_step/sec: 646.311
INFO:tensorflow:loss = 4.27678e-06, step = 23401 (0.154 sec)
INFO:tensorflow:global_step/sec: 625.039
INFO:tensorflow:loss = 4.15133e-06, step = 23501 (0.160 sec)
INFO:tensorflow:global_step/sec: 586.098
INFO:tensorflow:loss = 4.02964e-06, step = 23601 (0.170 sec)
INFO:tensorflow:global_step/sec: 653.767
INFO:tensorflow:loss = 3.91136e-06, step = 23701 (0.153 sec)
INFO:tensorflow:global_step/sec: 591.892
INFO:tensorflow:loss = 3.79655e-06, step = 23801 (0.170 sec)
INFO:tensorflow:global_step/sec: 625.962
INFO:tensorflow:loss = 3.68528e-06, step = 23901 (0.159 sec)
INFO:tensorflow:global_step/sec: 641.896
INFO:tensorflow:loss = 3.5772e-06, step = 24001 (0.156 sec)
INFO:tensorflow:global_step/sec: 625.436
INFO:tensorflow:loss = 3.47222e-06, step = 24101 (0.160 sec)
INFO:tensorflow:global_step/sec: 630.495
INFO:tensorflow:loss = 3.37042e-06, step = 24201 (0.159 sec)
INFO:tensorflow:global_step/sec: 592.421
INFO:tensorflow:loss = 3.2716e-06, step = 

INFO:tensorflow:global_step/sec: 568.048
INFO:tensorflow:loss = 3.85662e-07, step = 31501 (0.175 sec)
INFO:tensorflow:global_step/sec: 654.407
INFO:tensorflow:loss = 3.74422e-07, step = 31601 (0.153 sec)
INFO:tensorflow:global_step/sec: 632.07
INFO:tensorflow:loss = 3.63454e-07, step = 31701 (0.158 sec)
INFO:tensorflow:global_step/sec: 424.245
INFO:tensorflow:loss = 3.52739e-07, step = 31801 (0.236 sec)
INFO:tensorflow:global_step/sec: 715.246
INFO:tensorflow:loss = 3.42242e-07, step = 31901 (0.140 sec)
INFO:tensorflow:global_step/sec: 734.806
INFO:tensorflow:loss = 3.32394e-07, step = 32001 (0.136 sec)
INFO:tensorflow:global_step/sec: 749.745
INFO:tensorflow:loss = 3.22828e-07, step = 32101 (0.134 sec)
INFO:tensorflow:global_step/sec: 685.195
INFO:tensorflow:loss = 3.13467e-07, step = 32201 (0.145 sec)
INFO:tensorflow:global_step/sec: 635.731
INFO:tensorflow:loss = 3.04333e-07, step = 32301 (0.157 sec)
INFO:tensorflow:global_step/sec: 455.533
INFO:tensorflow:loss = 2.95352e-07, step =

INFO:tensorflow:global_step/sec: 708.533
INFO:tensorflow:loss = 3.48963e-08, step = 39601 (0.141 sec)
INFO:tensorflow:global_step/sec: 752.451
INFO:tensorflow:loss = 3.39685e-08, step = 39701 (0.133 sec)
INFO:tensorflow:global_step/sec: 750.714
INFO:tensorflow:loss = 3.30546e-08, step = 39801 (0.133 sec)
INFO:tensorflow:global_step/sec: 725.312
INFO:tensorflow:loss = 3.21517e-08, step = 39901 (0.137 sec)
INFO:tensorflow:global_step/sec: 643.895
INFO:tensorflow:loss = 3.1263e-08, step = 40001 (0.156 sec)
INFO:tensorflow:global_step/sec: 739.275
INFO:tensorflow:loss = 3.03873e-08, step = 40101 (0.135 sec)
INFO:tensorflow:global_step/sec: 732.034
INFO:tensorflow:loss = 2.95225e-08, step = 40201 (0.137 sec)
INFO:tensorflow:global_step/sec: 693.273
INFO:tensorflow:loss = 2.867e-08, step = 40301 (0.144 sec)
INFO:tensorflow:global_step/sec: 689.185
INFO:tensorflow:loss = 2.78372e-08, step = 40401 (0.145 sec)
INFO:tensorflow:global_step/sec: 735.538
INFO:tensorflow:loss = 2.70182e-08, step = 4

INFO:tensorflow:global_step/sec: 669.113
INFO:tensorflow:loss = 3.32637e-09, step = 47701 (0.149 sec)
INFO:tensorflow:global_step/sec: 653.547
INFO:tensorflow:loss = 3.24298e-09, step = 47801 (0.154 sec)
INFO:tensorflow:global_step/sec: 625.607
INFO:tensorflow:loss = 3.16054e-09, step = 47901 (0.159 sec)
INFO:tensorflow:global_step/sec: 677.066
INFO:tensorflow:loss = 3.07949e-09, step = 48001 (0.148 sec)
INFO:tensorflow:global_step/sec: 636.824
INFO:tensorflow:loss = 2.99847e-09, step = 48101 (0.157 sec)
INFO:tensorflow:global_step/sec: 623.738
INFO:tensorflow:loss = 2.91961e-09, step = 48201 (0.160 sec)
INFO:tensorflow:global_step/sec: 697.208
INFO:tensorflow:loss = 2.84153e-09, step = 48301 (0.143 sec)
INFO:tensorflow:global_step/sec: 683.347
INFO:tensorflow:loss = 2.76454e-09, step = 48401 (0.146 sec)
INFO:tensorflow:global_step/sec: 650.363
INFO:tensorflow:loss = 2.68827e-09, step = 48501 (0.154 sec)
INFO:tensorflow:global_step/sec: 654.325
INFO:tensorflow:loss = 2.61357e-09, step 

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x7f04317b7390>, 'feature_columns': [_RealValuedColumn(column_name='Weight', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _SparseColumnKeys(column_name='Species', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('Goblin', 'Human', 'ManBears'), num_oov_buckets=0, vocab_size=3, default_value=-1), combiner='sum', dtype=tf.string)], 'optimizer': None, 'gradient_clip_norm': None, 'joint_weights': False})

In [14]:
w_w = reg.get_variable_value('linear/Weight/weight')
print('Weight: {}'.format(w_w))

s_w = reg.get_variable_value('linear/Species/weights')
b = reg.get_variable_value('linear/bias_weight')
print('Species: {}'.format(s_w + b))

Weight: [[ 0.01000826]]
Species: [[ 0.89941633]
 [ 0.9994188 ]
 [ 1.09941947]]


In [1]:
import tflearn
import tflearn.datasets.mnist as mnist

from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

X, Y, X_test, Y_test = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

CNN = input_data(shape=[None,28,28,1], name='input')
CNN = conv_2d(CNN, 32, 5, activation='relu', regularizer='L2')
CNN = max_pool_2d(CNN, 2)
CNN = local_response_normalization(CNN)
CNN = conv_2d(CNN, 64, 5, activation='relu', regularizer='L2')
CNN = max_pool_2d(CNN, 2)
CNN = local_response_normalization(CNN)
CNN = fully_connected(CNN, 1024, activation=None)
CNN = dropout(CNN, 0.5)
CNN = fully_connected(CNN, 10, activation='softmax')
CNN = regression(CNN, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy', name='target')

model = tflearn.DNN(
    CNN, tensorboard_verbose=0, tensorboard_dir='MNIST_tflearn_board',
    checkpoint_path='MNIST_tflearn_checkpoints/checkpoint'
)

model.fit(
    {'input': X}, {'target': Y}, n_epoch=3, 
    validation_set=(
        {'input': X_test}, 
        {'target': Y_test}
    ),
    snapshot_step=1000, show_metric=True, run_id='convnet_mnist'
)

evaluation = model.evaluate({'input': X_test}, {'target': Y_test})
print(evaluation)

Training Step: 2579  | total loss: 0.05003 | time: 127.115s
| Adam | epoch: 003 | loss: 0.05003 - acc: 0.9841 -- iter: 54976/55000
Training Step: 2580  | total loss: 0.04802 | time: 132.409s
| Adam | epoch: 003 | loss: 0.04802 - acc: 0.9857 | val_loss: 0.04847 - val_acc: 0.9844 -- iter: 55000/55000
--
INFO:tensorflow:/code/MNIST_tflearn_checkpoints/checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/code/MNIST_tflearn_checkpoints/checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.
[0.98440000000000005]


In [4]:
import numpy as np
pred = model.predict({'input': X_test})
print((np.argmax(Y_test, 1) == np.argmax(pred, 1)).mean())

0.9844


### TF Learn

In [7]:
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
# Load data
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,
valid_portion=0.1)
X_train, Y_train = train
X_test, Y_test = test
# Sequence padding and converting labels to binary vectors
X_train = pad_sequences(X_train, maxlen=100, value=0.)
X_test = pad_sequences(X_test, maxlen=100, value=0.)
Y_train = to_categorical(Y_train, nb_classes=2)
Y_test = to_categorical(Y_test, nb_classes=2)
# Building an LSTM network
RNN = tflearn.input_data([None, 100])
RNN = tflearn.embedding(RNN, input_dim=10000, output_dim=128)
RNN = tflearn.lstm(RNN, 128, dropout=0.8)
RNN = tflearn.fully_connected(RNN, 2, activation='softmax')
RNN = tflearn.regression(RNN, optimizer='adam', learning_rate=0.001,
loss='categorical_crossentropy')
# Training the network
model = tflearn.DNN(RNN, tensorboard_verbose=0)
model.fit(X_train, Y_train, validation_set=(X_test, Y_test),
 show_metric=True, batch_size=32)

IndexError: list index out of range

In [8]:
from keras import backend as K

Using TensorFlow backend.


### LSTM

In [1]:
def to_one_hot(np_arr, num_classes):
    a = np.zeros((len(np_arr), num_classes))
    a[np.arange(len(np_arr)), np_arr] = 1
    return a    
    
def get_batches(batch_size, data_x, data_y):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    
    for start in range(0, len(instance_indices), batch_size):
        batch = instance_indices[start:start+batch_size]
        x = np.array(data_x)[batch]
        y = np.array(data_y)[batch]
        yield x, y
        
import tensorflow as tf
import numpy as np

lstm_size = 128
num_classes = 10
batch_size = 128
epochs = 10

with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, shape=(None, 28, 28), name='x')
    y = tf.placeholder(tf.int32, shape=(None, 10), name='y')

with tf.name_scope('rnn'):
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    
    outputs, (cell_state, states) = tf.nn.dynamic_rnn(
        lstm_cell, x, 
        dtype=tf.float32
    )
    state = states[-1]
        
with tf.name_scope('loss'):
    # Fully connected layer. 
    wy = tf.Variable(
        tf.truncated_normal(
            [lstm_size, num_classes], mean=0.0,
            stddev=.01, dtype=tf.float32
        ),
        name='wy'
    )
    bias = tf.Variable(
        tf.truncated_normal(
            [num_classes], mean=0.0,
            stddev=.01, dtype=tf.float32
        ),
        name='bias'
    )
    logits = tf.matmul(state, wy) + bias
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y,
        logits=logits,
        name='loss'
    ))
    correct = tf.equal(
        tf.argmax(y, 1), 
        tf.argmax(logits, 1)
    )
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
    
from tensorflow.keras.datasets.mnist import load_data

(x_train, y_train), (x_test, y_test) = load_data('/tmp/mnist.npz')    
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_one_hot(y_train, 10)
y_test = to_one_hot(y_test, 10)

x_train = x_train[:5000]
y_train = y_train[:5000]

with tf.Session() as sess:
    sess.run(tf.initializers.global_variables())
    
    for epoch in range(10):
        loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
            x: x_train, y: y_train
        })
        print('Loss, Accuracy at epoch %d: %f %f' % (epoch, loss, acc))
        
        x_ = np.reshape(x_train, (50, 100, 28, 28)) 
        y_ = np.reshape(y_train, (50, 100, 10))
        for x_batch, y_batch in zip(x_, y_):                
            _ = sess.run([train_step], feed_dict={
                x: x_batch, y: y_batch
            })
    loss, acc = sess.run([cross_entropy, accuracy], feed_dict={
        x: x_test, y: y_test
    })
    print('Loss, Accuracy at test: %f %f' % (loss, acc))

11493376/11490434 [==============================] - 42s 4us/step
Loss, Accuracy at epoch 0: 2.302670 0.101600
Loss, Accuracy at epoch 1: 1.366461 0.570000
Loss, Accuracy at epoch 2: 0.805893 0.750600
Loss, Accuracy at epoch 3: 0.555166 0.838000
Loss, Accuracy at epoch 4: 0.400283 0.881000
Loss, Accuracy at epoch 5: 0.443412 0.852200
Loss, Accuracy at epoch 6: 0.305160 0.906000
Loss, Accuracy at epoch 7: 0.220392 0.937400
Loss, Accuracy at epoch 8: 0.187605 0.947400
Loss, Accuracy at epoch 9: 0.163774 0.951600
Loss, Accuracy at test: 0.250608 0.925300


### Matrix multiplication example.

In [1]:
import tensorflow as tf

data = [
    [
        [1, 2],
        [1, 2],
        [1, 2]
    ],
    [
        [1, 2],
        [1, 2],
        [1, 2]
    ]
]

x = tf.Variable(data, dtype=tf.float32)
y = tf.Variable(data, dtype=tf.float32)

print(x)
print(y)

z = tf.matmul(x, tf.transpose(y, perm=(0, 2, 1)))

print(z)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    result = sess.run(z)
    print(result)

<tf.Variable 'Variable:0' shape=(2, 3, 2) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(2, 3, 2) dtype=float32_ref>
Tensor("MatMul:0", shape=(2, 3, 3), dtype=float32)
[[[ 5.  5.  5.]
  [ 5.  5.  5.]
  [ 5.  5.  5.]]

 [[ 5.  5.  5.]
  [ 5.  5.  5.]
  [ 5.  5.  5.]]]
